In [1]:
import torch
from transformers import GPT2Config, GPT2Model

import random
import numpy as np

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)

config = GPT2Config()
original_model = GPT2Model(config).cuda()
folded_model = GPT2Model(config).cuda()

folded_model.load_state_dict(original_model.state_dict())
original_model.eval()
folded_model.eval()

import utils
counter = utils.Counter()


In [2]:
hook_pre_fn, hook_fn = utils.create_analyse_hook_fns(counter)

input_ids = torch.randint(0, 1000, (1, 128)).cuda()
my_input_ids = utils.MetadataTensor(input_ids, centered=False).cuda()

with utils.HookManager(folded_model, hook_fn, hook_pre_fn):
    folded_model(my_input_ids)

print('LayerNorm:', counter.ln_cnt)
print('Foldable:', counter.foldable_cnt)
print('Center modules:', counter.center_modules)

 <  GPT2Model >
   wte : Embedding
   wpe : Embedding
   drop : Dropout
   h : ModuleList
   ln_f : LayerNorm
   <- MetadataTensor False (1, 128) 0 set()
   <  Embedding >
     <- MetadataTensor False (1, 128) 0 set()
     -> MetadataTensor True (1, 128, 768) 1 {Embedding(50257, 768)}
   </ Embedding >
   <  Embedding >
     <- Tensor None (1, 128) 0 set()
     -> MetadataTensor True (1, 128, 768) 1 {Embedding(1024, 768)}
   </ Embedding >
   <  Dropout >
     <- MetadataTensor True (1, 128, 768) 2 {Embedding(50257, 768), Embedding(1024, 768)}
     -> MetadataTensor True (1, 128, 768) 2 {Embedding(50257, 768), Embedding(1024, 768)}
   </ Dropout >
   <  GPT2Block >
     ln_1 : LayerNorm
     attn : GPT2SdpaAttention
     ln_2 : LayerNorm
     mlp : GPT2MLP
     <- MetadataTensor True (1, 128, 768) 2 {Embedding(50257, 768), Embedding(1024, 768)}
     <  LayerNorm >
       <- MetadataTensor True (1, 128, 768) 2 {Embedding(50257, 768), Embedding(1024, 768)}
       -> MetadataTensor True (

c:\Users\BobYu\miniconda3\envs\torch\Lib\site-packages\torch\_tensor.py:1437: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  ret = func(*args, **kwargs)


In [3]:
import modules

for layer in counter.layernorms:
    modules.replace_layer_norm_forward(layer)

for layer in counter.center_modules:
    modules.center_modules(layer)


In [4]:
output_queue = []
check = utils.Check()
replace = False

def hook_original(module, input, output):
    name = module.__class__.__name__
    output_queue.append((output, name))

    # if isinstance(output, tuple):
    #     output = output[0]

    # with torch._tensor_str.printoptions(precision=10, sci_mode=True):
    #     len_shape = len(output.shape)
    #     index = tuple([0] * (len_shape - 2) + [slice(None, 4), slice(None, 4)])
    #     print(module.__class__.__name__, output[index])

def check_close_and_replace(tensor_a, tensor_b, check: utils.Check, tensor_a_str, tensor_b_str):
    check.hide_val()
    locals()[tensor_a_str] = tensor_a
    locals()[tensor_b_str] = tensor_b
    if check.check_eq(tensor_a_str, tensor_b_str, abs_tol=1e-5, local_vars=locals()):
        if replace and isinstance(tensor_a, torch.Tensor) and isinstance(tensor_b, torch.Tensor):
            tensor_b.data = tensor_a.data
    check.show_val()

def apply_func_to_nested_tuple_pair(t1, t2, func, *args, **kwargs):
    if isinstance(t1, tuple) and isinstance(t2, tuple):
        return tuple(apply_func_to_nested_tuple_pair(x1, x2, func, *args, **kwargs) for x1, x2 in zip(t1, t2))
    else:
        return func(t1, t2, *args, **kwargs)

def hook_folded(module, input, output):
    folded_name = module.__class__.__name__ + '_folded'
    original_output, original_name = output_queue.pop(0)
    original_name += '_original'
    apply_func_to_nested_tuple_pair(original_output, output, check_close_and_replace, check, original_name, folded_name)

    # if isinstance(output, tuple):
    #     output0 = output[0]

    # with torch._tensor_str.printoptions(precision=10, sci_mode=True):
    #     len_shape = len(output0.shape)
    #     index = tuple([0] * (len_shape - 2) + [slice(None, 4), slice(None, 4)])
    #     print(module.__class__.__name__, output0[index])



In [5]:
with utils.HookManager(original_model, hook_original, None, list(original_model.modules())[1:]):
    original_out = original_model(input_ids)

with utils.HookManager(folded_model, hook_folded, None, list(folded_model.modules())[1:]):
    folded_out = folded_model(input_ids)


# 0 [ Test ] Embedding_original ?= Embedding_folded
Mean abs diff: 0.0005862714606337249
# 0 [ Fail ] Embedding_original != Embedding_folded

# 1 [ Test ] Embedding_original ?= Embedding_folded
Mean abs diff: 0.0006422363221645355
# 1 [ Fail ] Embedding_original != Embedding_folded

# 2 [ Test ] Dropout_original ?= Dropout_folded
Mean abs diff: 0.0008070007897913456
# 2 [ Fail ] Dropout_original != Dropout_folded

# 3 [ Test ] LayerNorm_original ?= SOLayerNorm_folded
Max diff: 0.024968624114990234
Location: [0, 88, 120]
LayerNorm_original: 4.170929908752441
SOLayerNorm_folded: 4.195898532867432
# 3 [ Fail ] LayerNorm_original != SOLayerNorm_folded

# 4 [ Test ] Conv1D_original ?= Conv1D_folded
Max diff: 0.014473915100097656
Location: [0, 108, 553]
Conv1D_original: 2.291804313659668
Conv1D_folded: 2.3062782287597656
# 4 [ Fail ] Conv1D_original != Conv1D_folded

# 5 [ Test ] Conv1D_original ?= Conv1D_folded
Max diff: 0.004483520984649658
Location: [0, 0, 682]
Conv1D_original: -0.1537477

In [6]:
check.check_eq('folded_out[0]', 'original_out[0]', local_vars=locals(), abs_tol=1e-5)

# 220 [ Test ] folded_out[0] ?= original_out[0]
=== folded_out[0] ===
tensor([[[2.4871e-02, -1.3620e-01, -3.4582e-01,  ..., 5.6413e-03, 6.5393e-01, -4.3367e-01],
         [5.1748e-01, 4.3117e-01, 7.8653e-01,  ..., -9.6986e-01, 8.7109e-01, -1.2818e+00],
         [7.8329e-01, 7.4768e-01, -2.5939e-01,  ..., 8.3396e-03, 1.0855e+00, -5.6037e-01],
         ...,
         [1.9191e-01, 7.3121e-01, -6.8771e-01,  ..., -2.5439e-01, -3.4247e-01, -2.6406e-01],
         [3.8035e-02, 6.7756e-01, -1.0164e+00,  ..., -1.9307e-01, 1.0875e+00, -6.1144e-01],
         [4.3601e-02, 2.6367e+00, -8.0250e-01,  ..., -1.9235e-01, 1.4889e+00, 7.4541e-01]]],
       device='cuda:0', grad_fn=<ViewBackward0>)
=== original_out[0] ===
tensor([[[2.1712e-02, -1.4150e-01, -3.4320e-01,  ..., 3.9669e-03, 6.5941e-01, -4.2921e-01],
         [5.1380e-01, 4.2810e-01, 7.8693e-01,  ..., -9.6909e-01, 8.7120e-01, -1.2768e+00],
         [7.8011e-01, 7.4773e-01, -2.6297e-01,  ..., 6.7078e-03, 1.0873e+00, -5.5771e-01],
         ...,
   

False

In [7]:
check.summary()

==== < Summary > ====
# 0 [ Fail ] Embedding_original != Embedding_folded (rel_tol=1e-05, abs_tol=1e-05)
# 1 [ Fail ] Embedding_original != Embedding_folded (rel_tol=1e-05, abs_tol=1e-05)
# 2 [ Fail ] Dropout_original != Dropout_folded (rel_tol=1e-05, abs_tol=1e-05)
# 3 [ Fail ] LayerNorm_original != SOLayerNorm_folded (rel_tol=1e-05, abs_tol=1e-05)
# 4 [ Fail ] Conv1D_original != Conv1D_folded (rel_tol=1e-05, abs_tol=1e-05)
# 5 [ Fail ] Conv1D_original != Conv1D_folded (rel_tol=1e-05, abs_tol=1e-05)
# 6 [ Fail ] Dropout_original != Dropout_folded (rel_tol=1e-05, abs_tol=1e-05)
# 7 [ Fail ] GPT2SdpaAttention_original != GPT2SdpaAttention_folded (rel_tol=1e-05, abs_tol=1e-05)
# 8 [ Fail ] GPT2SdpaAttention_original != GPT2SdpaAttention_folded (rel_tol=1e-05, abs_tol=1e-05)
# 9 [ Fail ] GPT2SdpaAttention_original != GPT2SdpaAttention_folded (rel_tol=1e-05, abs_tol=1e-05)
# 10 [ Pass ] GPT2SdpaAttention_original == GPT2SdpaAttention_folded (rel_tol=1e-05, abs_tol=1e-05)
# 11 [ Fail ] Laye

In [8]:
folded_model = GPT2Model(config).cuda()
original_model = GPT2Model(config).cuda()

folded_model.load_state_dict(original_model.state_dict())
folded_model.eval()
original_model.eval()

folded_counter = utils.Counter()
original_counter = utils.Counter()

folded_hook_pre_fn, folded_hook_fn = utils.create_analyse_hook_fns(folded_counter, _print=False)
original_hook_pre_fn, original_hook_fn = utils.create_analyse_hook_fns(original_counter, _print=False)

with utils.HookManager(folded_model, folded_hook_fn, folded_hook_pre_fn):
    folded_model(my_input_ids)

with utils.HookManager(original_model, original_hook_fn, original_hook_pre_fn):
    original_model(my_input_ids)

for layer in folded_counter.center_modules:
    modules.center_modules(layer)

for layer in folded_counter.layernorms:
    modules.replace_layer_norm_forward(layer, forward_fn=modules.soln_forward)

for layer in original_counter.layernorms:
    modules.replace_layer_norm_forward(layer, forward_fn=modules.myln_forward)


In [9]:
from torch.profiler import profile, record_function, ProfilerActivity, schedule

my_schedule = schedule(
    wait=100,
    warmup=50,
    active=250,
)

torch.cuda.empty_cache()

with torch.no_grad():
    with profile(
        activities=[
            ProfilerActivity.CPU, ProfilerActivity.CUDA
        ],
        schedule=my_schedule
    ) as prof:
        with record_function("folded_model_inference"):
            for _ in range(1200):
                folded_model(input_ids)
                prof.step()
    print(prof.key_averages().table())
    prof.export_chrome_trace("tmp/folded_trace.json")

    with profile(
        activities=[
            ProfilerActivity.CPU, ProfilerActivity.CUDA
        ],
        schedule=my_schedule
    ) as prof:
        with record_function("original_model_inference"):
            for _ in range(1200):
                original_model(input_ids)
                prof.step()
    print(prof.key_averages().table())
    prof.export_chrome_trace("tmp/original_trace.json")


-------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                    ProfilerStep*        49.87%       15.427s       100.00%       30.932s      41.243ms        7.369s        23.86%       30.883s      41.177ms           750  
                                       aten::view         1.38%     426.227ms         1.38%     426.227ms       3.789us        1.031s         3.34%        1.031s       9.162us        112500  
                                     at